In [1]:
!pip install langchain langchain-openai langchain-core langchain-community docx2txt pypdf sentence-transformers faiss-cpu


In [2]:
!pip install openai


In [3]:
!pip install --upgrade openai

In [4]:
!pip install -U langchain langchain-openai

  Using cached langchain_openai-1.1.6-py3-none-any.whl.metadata (2.6 kB)
  Using cached langchain_core-1.2.5-py3-none-any.whl.metadata (3.7 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.8/102.8 kB 3.2 MB/s eta 0:00:00
Using cached langchain_openai-1.1.6-py3-none-any.whl (84 kB)
Using cached langchain_core-1.2.5-py3-none-any.whl (484 kB)
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.81
    Uninstalling langchain-core-0.3.81:
      Successfully uninstalled langchain-core-0.3.81
  Attempting uninstall: langchain-openai
    Found existing installation: langchain-openai 0.3.35
    Uninstalling langchain-openai-0.3.35:
      Successfully uninstalled langchain-openai-0.3.35
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.27
    Uninstalling langchain-0.3.27:
      Successfully uninstalled langchain-0.3.27
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed.

In [1]:
import os
os.environ["OPENAI_API_KEY"] = "secret-key"
# print(os.environ["OPENAI_API_KEY"])  # This should print your API key (or part of it)


In [6]:
!pip install langsmith
import langsmith
!pip install -U langchain langchain-openai

In [7]:
os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_API_KEY"] = "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"
os.environ["LANGSMITH_PROJECT"] = "langchain-rag"

from langsmith import utils
utils.get_env_var.cache_clear()

# Now import and use LangChain or LangSmith SDK components that will trace


# ***Call LLM***


In [8]:
# from langchain_openai import ChatOpenAI

# llm = ChatOpenAI()
# llm.invoke("Hello, world!")

In [9]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-4o-mini")
llm_response = llm.invoke("Tell me a joke")

print(llm_response)


content='Why did the scarecrow win an award?\n\nBecause he was outstanding in his field!' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 11, 'total_tokens': 28, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_29330a9688', 'id': 'chatcmpl-Cr2JXwliQ75DFrYkNhVY0tsm7WM3G', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='lc_run--019b5ae1-f2bd-7722-888b-01dfa476fa1b-0' usage_metadata={'input_tokens': 11, 'output_tokens': 17, 'total_tokens': 28, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


# ***Output Parsers***







In [10]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()
chain = llm | output_parser
result = chain.invoke("Tell me a joke")
print(result)


Why did the scarecrow win an award?

Because he was outstanding in his field!


# ***Structured Output***

In [11]:
from typing import List
from pydantic import BaseModel, Field

class MobileReview(BaseModel):
        phone_model: str = Field(description="Name and model of the phone")
        rating: float = Field(description="Overall rating out of 5")
        pros: List[str] = Field(description="List of positive aspects")
        cons: List[str] = Field(description="List of negative aspects")
        summary: str = Field(description="Brief summary of the review")

review_text = """
    Just got my hands on the new Galaxy S21 and wow, this thing is slick! The screen is gorgeous,
    colors pop like crazy. Camera's insane too, especially at night - my Insta game's never been
    stronger. Battery life's solid, lasts me all day no problem.
    Not gonna lie though, it's pretty pricey. And what's with ditching the charger? C'mon Samsung.
    Also, still getting used to the new button layout, keep hitting Bixby by mistake.
    Overall, I'd say it's a solid 4 out of 5. Great phone, but a few annoying quirks keep it from
    being perfect. If you're due for an upgrade, definitely worth checking out!
    """

structured_llm = llm.with_structured_output(MobileReview)
output = structured_llm.invoke(review_text)
print(output)
print(output.pros)


phone_model='Samsung Galaxy S21' rating=4.0 pros=['Gorgeous display with vibrant colors', 'Insane camera performance, especially in low light', 'Solid battery life, lasts all day'] cons=['High price point', 'No charger included in the box', 'New button layout can be confusing, easily trigger Bixby'] summary='The Samsung Galaxy S21 impresses with its stunning screen and excellent camera performance, making it a solid choice for anyone looking to upgrade, despite some annoying quirks.'
['Gorgeous display with vibrant colors', 'Insane camera performance, especially in low light', 'Solid battery life, lasts all day']


# ***Prompt Templates***

In [12]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("Tell me a short joke about {topic}")
chain = prompt | llm | output_parser
result = chain.invoke({"topic": "programming"})
print(result)


Why do programmers prefer dark mode?

Because light attracts bugs!


# ***LLM Messages***

In [13]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
        SystemMessage(content="You are a helpful assistant that tells jokes."),
        HumanMessage(content="Tell me about programming")
    ]
response = llm.invoke(messages)
print(response)

# template = ChatPromptTemplate([
#         ("system", "You are a helpful assistant that tells jokes."),
#         ("human", "Tell me about {topic}")
#     ])
# chain = template | llm
# response = chain.invoke({"topic": "programming"})
# print(response)


content='Sure! Programming is the process of creating a set of instructions that a computer can follow to perform specific tasks. These instructions are typically written in programming languages like Python, Java, C++, and JavaScript, among others. \n\nNow, how about a programming joke to lighten the mood?\n\nWhy do programmers prefer dark mode?\n\nBecause light attracts bugs!' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 69, 'prompt_tokens': 24, 'total_tokens': 93, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_29330a9688', 'id': 'chatcmpl-Cr2Jc9227f9RW0rfHh9J5zj7ynGhw', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='lc_run--019b5ae2-0580-7fa2-bddc-86223d8ac8f8-0

In [14]:
template = ChatPromptTemplate([
        ("system", "You are a helpful assistant that tells jokes."),
        ("human", "Tell me about {topic}")
    ])
chain = template | llm
response = chain.invoke({"topic": "programming"})
print(response)

content='Certainly! Programming is the process of designing and building executable computer software to accomplish a specific task. It involves writing code in various programming languages like Python, Java, C++, and more. \n\nBut enough about the technical stuff! Here’s a programming joke for you:\n\nWhy do programmers prefer dark mode?\n\nBecause light attracts bugs! 🐜💻\n\nIf you want to know more about programming or need help with a specific concept, feel free to ask!' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 92, 'prompt_tokens': 24, 'total_tokens': 116, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_29330a9688', 'id': 'chatcmpl-Cr2Je7UrJGLRAP5deruzfaXHem7gd', 'service_tier': 'defa

## ***Loading Documents***

In [15]:
from langchain_community.document_loaders import PyPDFLoader, Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from typing import List
from langchain_core.documents import Document
import os

def load_documents(folder_path: str) -> List[Document]:
    documents = []
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        if filename.endswith('.pdf'):
            loader = PyPDFLoader(file_path)
        elif filename.endswith('.docx'):
            loader = Docx2txtLoader(file_path)
        else:
            print(f"Unsupported file type: {filename}")
            continue
        documents.extend(loader.load())
    return documents

folder_path = "/content/drive/MyDrive/Colab Notebooks/doc"
documents = load_documents(folder_path)
print(f"Loaded {len(documents)} documents from the folder.")


Loaded 5 documents from the folder.


# ***Splitting Documents***

In [16]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len
)

splits = text_splitter.split_documents(documents)
print(f"Split the documents into {len(splits)} chunks.")


Split the documents into 8 chunks.


In [17]:
print(documents[0])


page_content='Company: GreenFields BioTech

Headquarters: GreenFields BioTech is headquartered in Zurich, Switzerland. Known for its groundbreaking research in sustainable agriculture and biotechnology, the company has strategically positioned itself in Zurich, a city recognized for its leadership in scientific research and innovation. This location provides GreenFields BioTech with an ideal environment to collaborate with leading academic institutions and industry experts, driving forward its mission to create eco-friendly farming solutions.' metadata={'source': '/content/drive/MyDrive/Colab Notebooks/doc/Company_ GreenFields BioTech.docx'}


In [18]:
print(splits[3])

page_content='GreenGrow Innovations was founded in 2010 by Sarah Chen and Michael Rodriguez, two agricultural engineers with a passion for sustainable farming. The company started in a small garage in Portland, Oregon, with a simple mission: to make farming more environmentally friendly and efficient.



In its early days, GreenGrow focused on developing smart irrigation systems that could significantly reduce water usage in agriculture. Their first product, the WaterWise Sensor, was launched in 2012 and quickly gained popularity among local farmers. This success allowed the company to expand its research and development efforts.



By 2015, GreenGrow had outgrown its garage origins and moved into a proper office and research facility in the outskirts of Portland. This move coincided with the development of their second major product, the SoilHealth Monitor, which used advanced sensors to analyze soil composition and provide real-time recommendations for optimal crop growth.' metadata=

In [19]:
print(splits[0])


page_content='Company: GreenFields BioTech

Headquarters: GreenFields BioTech is headquartered in Zurich, Switzerland. Known for its groundbreaking research in sustainable agriculture and biotechnology, the company has strategically positioned itself in Zurich, a city recognized for its leadership in scientific research and innovation. This location provides GreenFields BioTech with an ideal environment to collaborate with leading academic institutions and industry experts, driving forward its mission to create eco-friendly farming solutions.' metadata={'source': '/content/drive/MyDrive/Colab Notebooks/doc/Company_ GreenFields BioTech.docx'}


In [20]:
print(splits[0].metadata)


{'source': '/content/drive/MyDrive/Colab Notebooks/doc/Company_ GreenFields BioTech.docx'}


# ***Using OpenAI Embeddings***

In [21]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()
document_embeddings = embeddings.embed_documents([split.page_content for split in splits])
print(f"Created embeddings for {len(document_embeddings)} document chunks.")


Created embeddings for 8 document chunks.


In [22]:
print(document_embeddings[4])

[-0.011857322417199612, -0.028335755690932274, 0.010321365669369698, -0.004197399597615004, -0.004995170049369335, 0.009063470177352428, -0.017425166442990303, -0.011029759421944618, -0.025316806510090828, -0.018471205607056618, 0.01685580238699913, 0.005657220724970102, -0.025965616106987, -0.008275629952549934, -0.0002608892391435802, 0.024535587057471275, 0.02815038152039051, -0.014803446829319, -0.024138357490301132, -0.0011734842555597425, -0.005733356345444918, 0.019795306026935577, -0.008249147795140743, 0.012704746797680855, 0.0069780112244188786, 0.016233475878834724, 0.0013166527496650815, -0.014538626186549664, 0.013267490081489086, 0.015266882255673409, -0.020099850371479988, -0.001251275185495615, -0.015770040452480316, 0.013611756265163422, -0.04282141849398613, -0.02738240361213684, 0.009275325573980808, 0.0057267360389232635, 0.007501030806452036, -0.00808363500982523, 0.029845232143998146, -0.01831231266260147, 0.020934032276272774, -0.010447154752910137, -0.0300570875

In [23]:
from langchain_community.embeddings.sentence_transformer import SentenceTransformerEmbeddings

embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
document_embeddings = embedding_function.embed_documents([split.page_content for split in splits])
print(document_embeddings[0][:5])  # Printing first 5 elements of the first embedding


/tmp/ipython-input-2990070485.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


[0.029335785657167435, -0.027797525748610497, -0.018463414162397385, -0.058924268931150436, 0.08951884508132935]


# ***Using SentenceTransformer***

In [24]:
from langchain_community.embeddings.sentence_transformer import SentenceTransformerEmbeddings

embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
document_embeddings = embedding_function.embed_documents([split.page_content for split in splits])
print(document_embeddings[0][:5])  # Printing first 5 elements of the first embedding


[0.029335785657167435, -0.027797525748610497, -0.018463414162397385, -0.058924268931150436, 0.08951884508132935]


# ***Creating the Vector Store***

In [25]:
!pip install langchain chromadb langchain-chroma

In [26]:
from langchain_chroma import Chroma

collection_name = "my_collection"
vectorstore = Chroma.from_documents(
    collection_name=collection_name,
    documents=splits,
    embedding=embedding_function,
    persist_directory="./chroma_db"
)
print("Vector store created and persisted to './chroma_db'")


Vector store created and persisted to './chroma_db'



# ***Performing Similarity Search***

In [27]:
query = "When was GreenGrow Innovations founded?"
search_results = vectorstore.similarity_search(query, k=2)
print(f"\nTop 2 most relevant chunks for the query: '{query}'\n")
for i, result in enumerate(search_results, 1):
    print(f"Result {i}:")
    print(f"Source: {result.metadata.get('source', 'Unknown')}")
    print(f"Content: {result.page_content}")
    print()



Top 2 most relevant chunks for the query: 'When was GreenGrow Innovations founded?'

Result 1:
Source: /content/drive/MyDrive/Colab Notebooks/doc/GreenGrow Innovations_ Company History.docx
Content: GreenGrow Innovations was founded in 2010 by Sarah Chen and Michael Rodriguez, two agricultural engineers with a passion for sustainable farming. The company started in a small garage in Portland, Oregon, with a simple mission: to make farming more environmentally friendly and efficient.



In its early days, GreenGrow focused on developing smart irrigation systems that could significantly reduce water usage in agriculture. Their first product, the WaterWise Sensor, was launched in 2012 and quickly gained popularity among local farmers. This success allowed the company to expand its research and development efforts.



By 2015, GreenGrow had outgrown its garage origins and moved into a proper office and research facility in the outskirts of Portland. This move coincided with the developmen

In [28]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 2})
retriever_results = retriever.invoke("When was GreenGrow Innovations founded?")
print(retriever_results)


[Document(id='ad7700b7-49fb-4a4c-a9fe-1b7217354e4a', metadata={'source': '/content/drive/MyDrive/Colab Notebooks/doc/GreenGrow Innovations_ Company History.docx'}, page_content='GreenGrow Innovations was founded in 2010 by Sarah Chen and Michael Rodriguez, two agricultural engineers with a passion for sustainable farming. The company started in a small garage in Portland, Oregon, with a simple mission: to make farming more environmentally friendly and efficient.\n\n\n\nIn its early days, GreenGrow focused on developing smart irrigation systems that could significantly reduce water usage in agriculture. Their first product, the WaterWise Sensor, was launched in 2012 and quickly gained popularity among local farmers. This success allowed the company to expand its research and development efforts.\n\n\n\nBy 2015, GreenGrow had outgrown its garage origins and moved into a proper office and research facility in the outskirts of Portland. This move coincided with the development of their sec

In [29]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

template = """Answer the question based only on the following context:
{context}
Question: {question}
Answer: """

prompt = ChatPromptTemplate.from_template(template)

def docs2str(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever | docs2str, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [30]:
question = "When was GreenGrow Innovations founded?"
response = rag_chain.invoke(question)
print(f"Question: {question}")
print(f"Answer: {response}")


Question: When was GreenGrow Innovations founded?
Answer: GreenGrow Innovations was founded in 2010.


# ***Creating a History-Aware Retriever***

In [31]:
%pip install -U langchain langchain-community langchain-openai


  Using cached langchain_community-0.4.1-py3-none-any.whl.metadata (3.0 kB)
  Using cached langchain_text_splitters-1.1.0-py3-none-any.whl.metadata (2.7 kB)
Using cached langchain_community-0.4.1-py3-none-any.whl (2.5 MB)
Using cached langchain_text_splitters-1.1.0-py3-none-any.whl (34 kB)
  Attempting uninstall: langchain-text-splitters
    Found existing installation: langchain-text-splitters 0.3.11
    Uninstalling langchain-text-splitters-0.3.11:
      Successfully uninstalled langchain-text-splitters-0.3.11
  Attempting uninstall: langchain-community
    Found existing installation: langchain-community 0.3.31
    Uninstalling langchain-community-0.3.31:
      Successfully uninstalled langchain-community-0.3.31


In [32]:
import langchain
print(langchain.__version__)
print(langchain.__file__)



1.2.0
/usr/local/lib/python3.12/dist-packages/langchain/__init__.py


In [37]:
%pip uninstall -y langchain
%pip uninstall -y langchain-core langchain-community langchain-openai


Found existing installation: langchain 0.3.27
Uninstalling langchain-0.3.27:
  Successfully uninstalled langchain-0.3.27
Found existing installation: langchain-core 0.3.81
Uninstalling langchain-core-0.3.81:
  Successfully uninstalled langchain-core-0.3.81
Found existing installation: langchain-community 0.3.31
Uninstalling langchain-community-0.3.31:
  Successfully uninstalled langchain-community-0.3.31
Found existing installation: langchain-openai 0.3.35
Uninstalling langchain-openai-0.3.35:
  Successfully uninstalled langchain-openai-0.3.35


In [38]:
%pip install -U "langchain<1.0" langchain-core langchain-community langchain-openai


  Using cached langchain-0.3.27-py3-none-any.whl.metadata (7.8 kB)
  Using cached langchain_core-1.2.5-py3-none-any.whl.metadata (3.7 kB)
  Using cached langchain_community-0.4.1-py3-none-any.whl.metadata (3.0 kB)
  Using cached langchain_openai-1.1.6-py3-none-any.whl.metadata (2.6 kB)
  Using cached langchain_core-0.3.81-py3-none-any.whl.metadata (3.2 kB)
INFO: pip is looking at multiple versions of langchain-community to determine which version is compatible with other requirements. This could take a while.
  Using cached langchain_community-0.4-py3-none-any.whl.metadata (3.0 kB)
  Using cached langchain_community-0.3.31-py3-none-any.whl.metadata (3.0 kB)
INFO: pip is looking at multiple versions of langchain-openai to determine which version is compatible with other requirements. This could take a while.
  Using cached langchain_openai-1.1.5-py3-none-any.whl.metadata (2.6 kB)
  Using cached langchain_openai-1.1.4-py3-none-any.whl.metadata (2.6 kB)
  Using cached langchain_openai-1.1

In [39]:
import langchain
print("langchain version:", langchain.__version__)
print("langchain path:", langchain.__file__)

import langchain.chains
print("✅ langchain.chains imported")


langchain version: 1.2.0
langchain path: /usr/local/lib/python3.12/dist-packages/langchain/__init__.py
✅ langchain.chains imported


In [43]:
import langchain
print(langchain.__version__)
print(langchain.__file__)


1.2.0
/usr/local/lib/python3.12/dist-packages/langchain/__init__.py


In [44]:
from langchain_core.prompts import MessagesPlaceholder
from langchain.chains.history_aware_retriever import create_history_aware_retriever
from langchain.chains.combine_documents import create_stuff_documents_chain


In [55]:
from langchain_core.prompts import MessagesPlaceholder
from langchain.chains import create_history_aware_retriever
from langchain.chains.combine_documents import create_stuff_documents_chain

contextualize_q_system_prompt = """
Given a chat history and the latest user question
which might reference context in the chat history,
formulate a standalone question which can be understood
without the chat history. Do NOT answer the question,
just reformulate it if needed and otherwise return it as is.
"""

contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

contextualize_chain = contextualize_q_prompt | llm | StrOutputParser()
print(contextualize_chain.invoke({"input": "Where is it headquartered?", "chat_history": []}))


What is the location of the company's headquarters?


In [48]:
from langchain.chains import create_retrieval_chain


In [56]:
history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt
)

qa_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful AI assistant. Use the following context to answer the user's question."),
    ("system", "Context: {context}"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{input}")
])

question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)
rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)


In [58]:
from langchain_core.messages import HumanMessage, AIMessage

chat_history = []
question1 = "When was GreenGrow Innovations founded?"
answer1 = rag_chain.invoke({"input": question1, "chat_history": chat_history})['answer']
chat_history.extend([
    HumanMessage(content=question1),
    AIMessage(content=answer1)
])

print(f"Human: {question1}")
print(f"AI: {answer1}\n")

question2 = "Where is it headquartered?"
answer2 = rag_chain.invoke({"input": question2, "chat_history": chat_history})['answer']
chat_history.extend([
    HumanMessage(content=question2),
    AIMessage(content=answer2)
])

print(f"Human: {question2}")
print(f"AI: {answer2}")


Human: When was GreenGrow Innovations founded?
AI: GreenGrow Innovations was founded in 2010.

Human: Where is it headquartered?
AI: GreenGrow Innovations is headquartered in California, and it also has offices in Iowa.


# ***Building a Multi-User Chatbot with SQLite Storage***

In [59]:
import sqlite3
from datetime import datetime
import uuid

DB_NAME = "rag_app.db"

def get_db_connection():
    conn = sqlite3.connect(DB_NAME)
    conn.row_factory = sqlite3.Row
    return conn

def create_application_logs():
    conn = get_db_connection()
    conn.execute('''CREATE TABLE IF NOT EXISTS application_logs
    (id INTEGER PRIMARY KEY AUTOINCREMENT,
    session_id TEXT,
    user_query TEXT,
    gpt_response TEXT,
    model TEXT,
    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP)''')
    conn.close()

def insert_application_logs(session_id, user_query, gpt_response, model):
    conn = get_db_connection()
    conn.execute('INSERT INTO application_logs (session_id, user_query, gpt_response, model) VALUES (?, ?, ?, ?)',
                 (session_id, user_query, gpt_response, model))
    conn.commit()
    conn.close()

def get_chat_history(session_id):
    conn = get_db_connection()
    cursor = conn.cursor()
    cursor.execute('SELECT user_query, gpt_response FROM application_logs WHERE session_id = ? ORDER BY created_at', (session_id,))
    messages = []
    for row in cursor.fetchall():
        messages.extend([
            {"role": "human", "content": row['user_query']},
            {"role": "ai", "content": row['gpt_response']}
        ])
    conn.close()
    return messages

# Initialize the database
create_application_logs()


In [60]:
# Example usage for a new user
session_id = str(uuid.uuid4())
question = "What is GreenGrow Innovations?"
chat_history = get_chat_history(session_id)
answer = rag_chain.invoke({"input": question, "chat_history": chat_history})['answer']
insert_application_logs(session_id, question, answer, "gpt-3.5-turbo")
print(f"Human: {question}")
print(f"AI: {answer}\n")

# Example of a follow-up question
question2 = "What was their first product?"
chat_history = get_chat_history(session_id)
answer2 = rag_chain.invoke({"input": question2, "chat_history": chat_history})['answer']
insert_application_logs(session_id, question2, answer2, "gpt-3.5-turbo")
print(f"Human: {question2}")
print(f"AI: {answer2}")


Human: What is GreenGrow Innovations?
AI: GreenGrow Innovations is a company that focuses on developing sustainable agricultural technologies. It gained national prominence in 2018 with the introduction of the EcoHarvest System, an integrated solution that combines smart irrigation, soil monitoring, and automated harvesting techniques. The company is headquartered with offices in California and Iowa, employs over 200 people, and is dedicated to promoting sustainable farming practices. GreenGrow is actively involved in ongoing projects related to vertical farming, drought-resistant crop development, and AI-powered farm management systems. Additionally, the company partners with universities and research institutions to advance agricultural technology and hosts annual conferences to share knowledge with farmers and industry professionals.

Human: What was their first product?
AI: GreenGrow Innovations' first product was the WaterWise Sensor, launched in 2012. This smart irrigation system